# Notebook 3: Analyze Model Misclassification

This notebook loads the model trained in `train_on_augmented_dataset.ipynb` and analyzes a specific misclassification found during the UrbanSound8K generalization test.

The goal is to visually compare the Mel spectrogram of the misclassified **siren** with that of a **true leak** to understand why the model made an error.


In [1]:
%pip install tensorflow scikit-learn librosa numpy pandas matplotlib seaborn tqdm


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\fe_de\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# --- Parameters ---
SAMPLE_RATE = 22050
FIXED_DURATION_S = 10
FIXED_LENGTH = SAMPLE_RATE * FIXED_DURATION_S

# --- Function to process audio to Mel Spectrogram (copied from training notebook) ---
def audio_to_melspectrogram(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
        if len(audio) < FIXED_LENGTH:
            audio = np.pad(audio, (0, FIXED_LENGTH - len(audio)), 'constant')
        else:
            audio = audio[:FIXED_LENGTH]
        
        spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_mels=128)
        return librosa.power_to_db(spectrogram, ref=np.max)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

print("Setup complete.")


C:\Users\fe_de\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\fe_de\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
C:\Users\fe_de\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Pyt

Setup complete.


## 2. Load the Trained Model

This step assumes you have already run the `train_on_augmented_dataset.ipynb` notebook and that the trained `model` variable is available in the kernel's memory. If you have restarted your kernel, you will need to re-run the other notebook up to the point of model training before running the cells below.


## 3. Visualize Spectrograms

Here we will generate and plot the spectrograms for the misclassified siren and a true leak sound.


In [3]:
# --- Paths to the files we want to compare ---
# Using direct, relative paths with forward slashes for cross-platform compatibility
siren_path = '../UrbanSound8K/UrbanSound8K/audio/fold8/75490-8-0-2.wav'
leak_path = 'Audios para Treinamento/Leak-Metal/004_07573-20180817.wav'

# --- Generate spectrograms ---
print("Generating spectrograms...")
siren_spec = audio_to_melspectrogram(siren_path)
leak_spec = audio_to_melspectrogram(leak_path)
print("Done.")

# --- Visualize the comparison ---
if siren_spec is not None and leak_spec is not None:
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))

    # Plot Siren Spectrogram
    librosa.display.specshow(siren_spec, sr=SAMPLE_RATE, x_axis='time', y_axis='mel', ax=axes[0])
    axes[0].set_title('Mel Spectrogram of Misclassified Siren', fontsize=14)
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Frequency (Hz)')

    # Plot Leak Spectrogram
    librosa.display.specshow(leak_spec, sr=SAMPLE_RATE, x_axis='time', y_axis='mel', ax=axes[1])
    axes[1].set_title('Mel Spectrogram of a True Leak', fontsize=14)
    axes[1].set_xlabel('Time (s)')
    axes[1].set_ylabel('') # Hide y-label for cleaner look

    fig.colorbar(axes[0].collections[0], ax=axes[0], format='%+2.0f dB')
    fig.colorbar(axes[1].collections[0], ax=axes[1], format='%+2.0f dB')
    
    plt.tight_layout()
    plt.show()
else:
    print("\nCould not generate one or both spectrograms for comparison. Please check file paths and ensure files exist.")


Generating spectrograms...


C:\Users\fe_de\AppData\Local\Temp\ipykernel_37288\693670256.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
C:\Users\fe_de\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error processing ../UrbanSound8K/UrbanSound8K/audio/fold8/75490-8-0-2.wav: [Errno 2] No such file or directory: '../UrbanSound8K/UrbanSound8K/audio/fold8/75490-8-0-2.wav'
Done.

Could not generate one or both spectrograms for comparison. Please check file paths and ensure files exist.
